In [1]:
import sys
print (sys.version)

3.7.2 (v3.7.2:9a3ffc0492, Dec 24 2018, 02:44:43) 
[Clang 6.0 (clang-600.0.57)]


In [7]:
#https://stackoverflow.com/questions/47944782/r-data-scraping-crawling-with-dynamic-multiple-urls
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [186]:
import locale, datetime #for stringparsing in german
locale.setlocale(locale.LC_ALL, "de_de")
#locale.setlocale(locale.LC_ALL, "fr_FR")

'de_de'

**Extracting the index leading to mandate**

In [33]:
#highlight_docid=atf%3A%2F%2F95-IV-162
queryKeyword = "mord"
queryRank = "1"
#bgerQuery2 = "https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=1954&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words="+ queryKeyword + "&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank="+queryRank+"&highlight_docid=atf%3A%2F%2F95-IV-162%3Ade&number_of_ranks=945&azaclir=clir"

In [34]:
#bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year=1954&to_year=2019'

In [35]:
from_year = "2017"
to_year = "2019"

bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year='+ from_year + '&to_year=' + to_year

In [36]:
bgerQuery2

'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words=mord&lang=de&top_subcollection_clir=bge&from_year=2017&to_year=2019'

In [37]:
#check if content was loaded properly
bgerQuery2 = requests.get(bgerQuery2)
bgerQuery2

<Response [200]>

In [38]:
#bgerQuery2.content

In [39]:
#soup = BeautifulSoup(page.content, 'html.parser')

In [40]:
bgerSoup = BeautifulSoup(bgerQuery2.content, 'html.parser')

In [41]:
bgerSoup


<!DOCTYPE html>

<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width initial-scale=1" name="viewport"/>
<link href="/modules/tfl-internet-templates/images/favicon.ico" rel="icon"/>
<title>Eurospider Suche: mord</title>
<link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/master.css" rel="stylesheet" title="Eurospider Default Screen Style" type="text/css"/>
<link href="/modules/tfl-internet-templates/css/styles.css" media="screen" rel="stylesheet" type="text/css"/>
<style media="print" type="text/css">
        @media print{
            .body .size1of3{
                display: none;
            }
            .head .grid_left{
                display: none;
            }
            .access-a, .form-search, .define-admin {
                display: none !important;
            }
            body{
                font-family: Arial, sans-serif;
                font-size: 12px !important;
       

In [42]:
links = bgerSoup.find_all(class_='rank_title')
linksList = []
for a in links:
    linksList.append(a.find('a')['href'])
    #print(a)
print(linksList[0])

https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=1&highlight_docid=atf%3A%2F%2F144-IV-285%3Ade&number_of_ranks=8&azaclir=clir


**Make a new query with Linkslist (only based on first entry)**

In [43]:
mBger = requests.get(linksList[0])

In [44]:
#mBger.content

In [45]:
mBger = BeautifulSoup(mBger.content, 'html.parser')

In [46]:
#mBger

**display children / find correct html elements**

In [47]:
len(list(mBger.children))
#find body
body = list(mBger.children)[12]
len(list(body.children))
#body

33

**get all required variables from verdict**

**dateparser for string dates**

In [167]:
text2 = "41. Urteil des Kassationshofes vom 5. octobre 1969 i.S. Staatsanwaltschaft des Kantons Aargau gegen Brunner."

def parseDate(text):
    pattern = re.compile(r"""
        \d\d?   # one or two digits
        \.?
        \s?      # \s for space - one space after
        [a-z,ä,ö,ü]+  # at least one+ ascii letters (ignore case is use)set of characters that you wish to match / since + means ‘one or more repetitions’
        \s?      # one space after
        \d{4}   # four decimal digits = \d (year)
    """,re.IGNORECASE|re.VERBOSE)
    if (pattern.search(text)!= None):
        #print(pattern.search(text).group(), type(pattern.search(text).group()))
        dateStr = pattern.search(text).group()
        dateStr = dateStr.replace(".", "")
        #dateStr = dateStr.format('DD MMMM YYYY', locale='de')
        dateStr = datetime.datetime.strptime(dateStr, "%d %B %Y")
        #dateStr = dateStr.strftime('%m/%d/%Y')
        return dateStr
    else:
        return 0 #make 0 if date can't be found ig not existent or different language

In [197]:
import arrow
dateTest = '5 mars 1969'
try:
    aDate = arrow.get(dateTest, 'D MMMM YYYY', locale='de')
    aDate.format('YYYY-MMMM-DD')
    print("1",aDate)
except:
    aDate = arrow.get(dateTest, 'D MMMM YYYY', locale='fr')
    aDate.format('YYYY-MMMM-DD')
    print("2",aDate)
    
#parseDate(text2)

2 1969-03-05T00:00:00+00:00


In [168]:
parseDate("3 Juli 2018")

datetime.datetime(2018, 7, 3, 0, 0)

In [85]:
#ID of mandate
mID = body.find_all('div', class_="center pagebreak")[0].get_text()[:13]
#mandID

#referenced mandates
#without link
mRefs = body.find_all(class_="bgeref_err")
#with link
mRefsLinked = body.find_all(class_="bgeref_id")

#date:
mDate = body.find_all(class_="paraatf")[0].get_text()
#mDate = parseDate(mDate)
mDate = parseDate(mDate)
mDate

0

**Get NAMES, DATE and SUBLINKS of each linked Mandate in list (PROBLEM: many are in french & dateparser return 0)**

In [93]:
def listing(elementlist):
    ar = []
    for item in elementlist:
        #print(item.get_text())
        ar.append(item.get_text())
    #print(ar)
    return ar

In [94]:
#names of referenced mandates
mRefs = listing(mRefs)
mRefsLinked_names = listing(mRefsLinked)
#mRefsLinked_names

In [86]:
#links of liked reference mandates
linksListref = []
for a in mRefsLinked:
    linksListref.append(a['href'])

In [138]:
linksListref

['https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=0&highlight_docid=atf%3A%2F%2F144-IV-1%3Ade&number_of_ranks=0&azaclir=clir#page1',
 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=0&highlight_docid=atf%3A%2F%2F141-IV-155%3Ade&number_of_ranks=0&azaclir=clir#page155',
 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&ins

In [135]:
for i,val in enumerate(linksListref):
    #print(i,val)
    r = requests.get(val)
    linkSoup = BeautifulSoup(r.content, 'html.parser')
    body = list(linkSoup.children)[12]
    #ID of mandate
    mID = body.find_all('div', class_="center pagebreak")[0].get_text()[:13]
    #referenced mandates
    #without link
    mRefs = listing(body.find_all(class_="bgeref_err"))
    #with link
    mRefsLinked = listing(body.find_all(class_="bgeref_id"))

    #date:
    mDate = body.find_all(class_="paraatf")[0].get_text()
    #mDate = parseDate(mDate)
    mDate = parseDate(mDate)
    print(i,mID, mRefs, mRefsLinked, mDate)

0 BGE 144 IV 1  [] ['ATF 138 I 97', 'ATF 142 IV 383', 'ATF 142 IV 383', 'ATF 141 IV 155', 'ATF 140 IV 57', 'ATF 139 IV 209', 'ATF 137 IV 305', 'ATF 141 IV 305', 'ATF 141 IV 155', 'ATF 140 IV 57', 'ATF 137 IV 79', 'ATF 125 IV 4', 'ATF 120 IV 365', 'ATF 103 IV 142', 'ATF 100 IV 263', 'ATF 100 IV 104', 'ATF 140 IV 57', 'ATF 140 IV 57', 'ATF 130 III 248', 'ATF 112 III 90', 'ATF 138 III 497', 'ATF 141 IV 317', 'ATF 141 IV 305'] 0
1 BGE 141 IV 15 [] ['BGE 137 IV 305', 'BGE 129 IV 107', 'BGE 129 IV 305', 'BGE 137 IV 305', 'BGE 125 IV 4', 'BGE 137 IV 79', 'BGE 134 IV 185', 'BGE 128 IV 145', 'BGE 104 IV 175', 'BGE 108 IV 41', 'BGE 101 IV 312', 'BGE 101 IV 312', 'BGE 104 IV 175', 'BGE 85 IV 139', 'BGE 137 II 431', 'BGE 137 IV 305', 'BGE 117 IV 233'] 0
2 BGE 140 IV 57 [] ['ATF 139 III 133', 'ATF 138 I 435', 'ATF 137 IV 122', 'ATF 128 II 259', 'ATF 138 IV 186', 'ATF 137 IV 340', 'ATF 129 I 103', 'ATF 128 I 129', 'ATF 131 I 333', 'ATF 131 I 425', 'ATF 137 III 522', 'ATF 136 IV 92', 'ATF 126 I 97', 

29 BGE 137 IV 79 [] ['DTF 129 IV 238'] 0
30 BGE 142 IV 38 [] [] 0


In [53]:
"id", mID, "referenced mandates", mRefs, "referenced linked mandates", mRefsLinked, "date", mDate

('id',
 'BGE 144 IV 28',
 'referenced mandates',
 [],
 'referenced linked mandates',
 ['BGE 144 IV 1',
  'BGE 141 IV 155',
  'BGE 140 IV 57',
  'BGE 141 IV 155',
  'BGE 129 IV 305',
  'BGE 144 IV 1',
  'BGE 141 IV 155',
  'BGE 140 IV 57',
  'BGE 138 IV 1',
  'BGE 136 IV 4',
  'BGE 129 II 453',
  'BGE 141 IV 155',
  'BGE 138 IV 1',
  'BGE 136 IV 4',
  'BGE 144 IV 1',
  'BGE 137 IV 79',
  'BGE 125 IV 4',
  'BGE 120 IV 365',
  'BGE 141 IV 155',
  'BGE 137 IV 305',
  'BGE 125 IV 4',
  'BGE 132 III 315',
  'BGE 132 III 305',
  'BGE 139 IV 209',
  'BGE 143 II 661',
  'BGE 143 III 242',
  'BGE 142 IV 237',
  'BGE 137 IV 79',
  'BGE 137 IV 79',
  'BGE 137 IV 79',
  'BGE 142 IV 383'],
 'date',
 0)

**Save variables in dynamic dictionary**

In [82]:
#l=['mID','mRefs','mRefsLinked','links','date']
#objs = [mID,mRefs, mRefsLinked, linksListref, mDate]
l=['mID','mRefs','mRefsLinked','date']
objs = [mID,mRefs, mRefsLinked, mDate]
mDict = {}
for i, val in enumerate(l):
    k= objs[i]
    mDict[val]=k
print(mDict, type(mDict))

{'mID': 'BGE 144 IV 28', 'mRefs': [], 'mRefsLinked': ['BGE 144 IV 1', 'BGE 141 IV 155', 'BGE 140 IV 57', 'BGE 141 IV 155', 'BGE 129 IV 305', 'BGE 144 IV 1', 'BGE 141 IV 155', 'BGE 140 IV 57', 'BGE 138 IV 1', 'BGE 136 IV 4', 'BGE 129 II 453', 'BGE 141 IV 155', 'BGE 138 IV 1', 'BGE 136 IV 4', 'BGE 144 IV 1', 'BGE 137 IV 79', 'BGE 125 IV 4', 'BGE 120 IV 365', 'BGE 141 IV 155', 'BGE 137 IV 305', 'BGE 125 IV 4', 'BGE 132 III 315', 'BGE 132 III 305', 'BGE 139 IV 209', 'BGE 143 II 661', 'BGE 143 III 242', 'BGE 142 IV 237', 'BGE 137 IV 79', 'BGE 137 IV 79', 'BGE 137 IV 79', 'BGE 142 IV 383'], 'date': 0} <class 'dict'>


In [83]:
mRefsLinked.append(mID)
mRefsLinked

['BGE 144 IV 1',
 'BGE 141 IV 155',
 'BGE 140 IV 57',
 'BGE 141 IV 155',
 'BGE 129 IV 305',
 'BGE 144 IV 1',
 'BGE 141 IV 155',
 'BGE 140 IV 57',
 'BGE 138 IV 1',
 'BGE 136 IV 4',
 'BGE 129 II 453',
 'BGE 141 IV 155',
 'BGE 138 IV 1',
 'BGE 136 IV 4',
 'BGE 144 IV 1',
 'BGE 137 IV 79',
 'BGE 125 IV 4',
 'BGE 120 IV 365',
 'BGE 141 IV 155',
 'BGE 137 IV 305',
 'BGE 125 IV 4',
 'BGE 132 III 315',
 'BGE 132 III 305',
 'BGE 139 IV 209',
 'BGE 143 II 661',
 'BGE 143 III 242',
 'BGE 142 IV 237',
 'BGE 137 IV 79',
 'BGE 137 IV 79',
 'BGE 137 IV 79',
 'BGE 142 IV 383',
 'BGE 144 IV 28']

In [55]:
#https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
#pd.DataFrame.from_dict(mDict, orient='index')

In [71]:
ids = ['BGE 144 IV 28','BGE 144 IV 1']
dates = ['1969-12-5', '1969-9-4']
relevancy = [10, 4]

itemsN = ["id", "date", "relevancy"]
nodes = []

itemsL = ["source", "target"]
links = []

for i, val in enumerate(ids):
    print(i, val)
    oj = {"id": val, "date":dates[i], "relevancy":relevancy[i]}
    nodes.append(oj)
    print(nodes)

    
data= {
    'nodes' : nodes,
    'links': 0
}

print(data)

0 BGE 144 IV 28
[{'id': 'BGE 144 IV 28', 'date': '1969-12-5', 'relevancy': 10}]
1 BGE 144 IV 1
[{'id': 'BGE 144 IV 28', 'date': '1969-12-5', 'relevancy': 10}, {'id': 'BGE 144 IV 1', 'date': '1969-9-4', 'relevancy': 4}]
{'nodes': [{'id': 'BGE 144 IV 28', 'date': '1969-12-5', 'relevancy': 10}, {'id': 'BGE 144 IV 1', 'date': '1969-9-4', 'relevancy': 4}], 'lins': 0}
